<a href="https://colab.research.google.com/github/TheMihirNaik/google-search-console-api-using-python/blob/main/Visualize_GSC_Data_using_Plotly%2C_Unique_Query_Count_%26_URL_Count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required python packages
!pip install oauth2client
!pip install google-api-python-client
!pip install httplib2

In [ ]:
# Import required packages
from oauth2client.client import OAuth2WebServerFlow
from googleapiclient.discovery import build
import httplib2

# Google Cloud Project Client ID & Client Secrets
CLIENT_ID = "Enter your own CLIENT_ID"
CLIENT_SECRET = "Enter your own CLIENT_SECRET"
OAUTH_SCOPE = "https://www.googleapis.com/auth/webmasters.readonly"
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print("Go to the following link in your browser: " + authorize_url)
auth_code = input("Enter your Authorization Code here:")
credentials = flow.step2_exchange(auth_code)
http = httplib2.Http()
creds = credentials.authorize(http)
webmasters_service = build('searchconsole', 'v1', http=creds)

Go to the following link in your browser: https://accounts.google.com/o/oauth2/v2/auth?client_id=1085687348720-cpabg6ttpsqjji0j6pgbkb8sbcdcp4nt.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fwebmasters.readonly&access_type=offline&response_type=code
Enter your Authorization Code here:4/1AVHEtk4sAKLU9vRYnn0e2MGR9JeFtedqldjp1sjYOzQE4L62A1UoQC_5Fvc


# List of Properties in GSC Account

In [ ]:
# Get a list of site in my Google Search Console Account
site_list = webmasters_service.sites().list().execute()

site_list

{'siteEntry': [{'siteUrl': 'https://moneymonkdigital.com/',
   'permissionLevel': 'siteOwner'},
  {'siteUrl': 'https://www.monocubed.com/', 'permissionLevel': 'siteFullUser'},
  {'siteUrl': 'https://www.tridentpbi.in/',
   'permissionLevel': 'siteFullUser'}]}

# Getting GSC Data & Preparing Dataframe using Pandas

In [ ]:
website = 'https://www.monocubed.com/'


# create an empty list to store the rows from response
all_responses = []

# define a startRow
startRow = 0

while (startRow == 0) or (startRow%25000 == 0):
  # build a request body
  request_body = {
      "startDate"	: '2022-06-01',
      "endDate" : '2023-03-23',
      "dimensions" : ['DATE', 'PAGE'],
      "dimensionFilterGroups": [ { "filters": [
            {"dimension": "COUNTRY", "expression": "USA", "operator": "equals" }
            ]}],
      "rowLimit" : 25000,
      "dataState" : "final",
      'startRow' : startRow
      }

  #get gsc response
  response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

  #update the rows
  startRow = startRow + len(response_data['rows'])
  print("fetched up to " + str(startRow) + " rows of data")

  # for loop tos save all the rows in all_responses
  for row in response_data['rows']:
    all_responses.append(row)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 121826 rows of data


In [ ]:
len(all_responses)

121826

In [ ]:
all_responses[0]

{'keys': ['2023-01-26',
  'https://www.monocubed.com/blog/best-front-end-frameworks/'],
 'clicks': 73,
 'impressions': 265,
 'ctr': 0.27547169811320754,
 'position': 12.679245283018869}

In [ ]:
#empty list to build a dataframe
data_for_df = []

for each in all_responses:

  temp = []

  #date
  temp.append(each['keys'][0])

  #page
  temp.append(each['keys'][1])

  #clicks
  temp.append(each['clicks'])

  #impressions
  temp.append(each['impressions'])

  #ctr
  temp.append(each['ctr'])

  #position
  temp.append(each['position'])

  data_for_df.append(temp)



In [ ]:
data_for_df[0]

['2023-01-26',
 'https://www.monocubed.com/blog/best-front-end-frameworks/',
 73,
 265,
 0.27547169811320754,
 12.679245283018869]

In [ ]:
import pandas as pd

df = pd.DataFrame(data_for_df, columns=['date', 'page', 'clicks', 'impressions', 'ctr', 'position'])

In [ ]:
df

,date,page,clicks,impressions,ctr,position
0,2023-01-26,https://www.monocubed.com/blog/best-front-end-...,73,265,0.275472,12.679245
1,2022-11-28,https://www.monocubed.com/blog/best-front-end-...,71,286,0.248252,12.108392
2,2022-09-13,https://www.monocubed.com/blog/best-front-end-...,66,286,0.230769,12.503497
3,2022-09-14,https://www.monocubed.com/blog/best-front-end-...,66,281,0.234875,9.149466
4,2022-09-20,https://www.monocubed.com/blog/best-front-end-...,66,292,0.226027,10.383562
...,...,...,...,...,...,...
121821,2023-03-23,https://www.monocubed.com/terms-of-use/,0,1,0.000000,36.000000
121822,2023-03-23,https://www.monocubed.com/testimonials/,0,1,0.000000,4.000000
121823,2023-03-23,https://www.monocubed.com/vue/,0,6,0.000000,48.833333
121824,2023-03-23,https://www.monocubed.com/web-portal-guides/,0,3,0.000000,23.333333


# Grouped By Date - Simple GSC Line Chart

In [ ]:
#group by date

df_grouped_by_date = df.groupby("date").agg(
    clicks = ('clicks', 'sum'),
    impressions = ('impressions', 'sum'),
    position = ('position', 'mean')
    ).reset_index()

In [ ]:
df_grouped_by_date

,date,clicks,impressions,position
0,2022-06-01,138,8751,16.275233
1,2022-06-02,168,9098,15.417319
2,2022-06-03,152,9073,15.040699
3,2022-06-04,83,7569,16.179336
4,2022-06-05,86,8083,15.985340
...,...,...,...,...
291,2023-03-19,62,10213,22.922653
292,2023-03-20,113,11999,20.392908
293,2023-03-21,102,11429,19.709316
294,2023-03-22,90,11855,19.705558


In [ ]:
import plotly.express as px

fig = px.line(df_grouped_by_date, x="date", y="clicks")

fig.show()

# Grouped by Date & Page : Count of Pages : Create Charts

In [ ]:
count_pages_df = df.groupby("date").agg(
    clicks = ('clicks', 'sum'),
    impressions = ('impressions', 'sum'),
    position = ('position', 'mean'),
    count_pages = ('page', 'count')
    ).reset_index()

count_pages_df

,date,clicks,impressions,position,count_pages
0,2022-06-01,138,8751,16.275233,365
1,2022-06-02,168,9098,15.417319,382
2,2022-06-03,152,9073,15.040699,385
3,2022-06-04,83,7569,16.179336,369
4,2022-06-05,86,8083,15.985340,379
...,...,...,...,...,...
291,2023-03-19,62,10213,22.922653,389
292,2023-03-20,113,11999,20.392908,400
293,2023-03-21,102,11429,19.709316,408
294,2023-03-22,90,11855,19.705558,401


In [ ]:
import plotly.express as px

fig = px.line(count_pages_df, x="date", y="count_pages", title="No. of Unique Pages Ranking in Google in USA")

fig.show()

# Grouped by Date and Count of Queries

In [ ]:
website = 'https://www.monocubed.com/'


# create an empty list to store the rows from response
all_responses = []

# define a startRow
startRow = 0

while (startRow == 0) or (startRow%25000 == 0):
  # build a request body
  request_body = {
      "startDate"	: '2022-06-01',
      "endDate" : '2023-03-23',
      "dimensions" : ['DATE', 'QUERY'],
      "dimensionFilterGroups": [ { "filters": [
            {"dimension": "COUNTRY", "expression": "USA", "operator": "equals" }
            ]}],
      "rowLimit" : 25000,
      "dataState" : "final",
      'startRow' : startRow
      }

  #get gsc response
  response_data = webmasters_service.searchanalytics().query(siteUrl=website, body=request_body).execute()

  #update the rows
  startRow = startRow + len(response_data['rows'])
  print("fetched up to " + str(startRow) + " rows of data")

  # for loop tos save all the rows in all_responses
  for row in response_data['rows']:
    all_responses.append(row)

fetched up to 25000 rows of data
fetched up to 50000 rows of data
fetched up to 75000 rows of data
fetched up to 100000 rows of data
fetched up to 125000 rows of data
fetched up to 150000 rows of data
fetched up to 175000 rows of data
fetched up to 200000 rows of data
fetched up to 225000 rows of data
fetched up to 250000 rows of data
fetched up to 275000 rows of data
fetched up to 300000 rows of data
fetched up to 325000 rows of data
fetched up to 350000 rows of data
fetched up to 375000 rows of data
fetched up to 400000 rows of data
fetched up to 425000 rows of data
fetched up to 450000 rows of data
fetched up to 475000 rows of data
fetched up to 500000 rows of data
fetched up to 525000 rows of data
fetched up to 550000 rows of data
fetched up to 575000 rows of data
fetched up to 600000 rows of data
fetched up to 625000 rows of data
fetched up to 650000 rows of data
fetched up to 675000 rows of data
fetched up to 700000 rows of data
fetched up to 725000 rows of data
fetched up to 750

In [ ]:
#empty list to build a dataframe
data_for_df = []

for each in all_responses:

  temp = []

  #date
  temp.append(each['keys'][0])

  #query
  temp.append(each['keys'][1])

  #clicks
  temp.append(each['clicks'])

  #impressions
  temp.append(each['impressions'])

  #ctr
  temp.append(each['ctr'])

  #position
  temp.append(each['position'])

  data_for_df.append(temp)

In [ ]:
import pandas as pd

df = pd.DataFrame(data_for_df, columns=['date', 'query', 'clicks', 'impressions', 'ctr', 'position'])

In [ ]:
count_queries_df = df.groupby("date").agg(
    clicks = ('clicks', 'sum'),
    impressions = ('impressions', 'sum'),
    position = ('position', 'mean'),
    count_queries = ('query', 'count')
    ).reset_index()

count_queries_df

,date,clicks,impressions,position,count_queries
0,2022-06-01,138,6079,37.060065,2967
1,2022-06-02,167,6125,36.096947,2981
2,2022-06-03,152,6320,37.264146,3160
3,2022-06-04,83,5351,39.679963,2950
4,2022-06-05,86,5843,39.792357,2989
...,...,...,...,...,...
291,2023-03-19,62,8618,44.853651,4346
292,2023-03-20,113,10085,41.481104,4522
293,2023-03-21,102,9618,40.998940,4511
294,2023-03-22,90,9961,40.605792,4505


In [ ]:
import plotly.express as px

fig = px.line(count_queries_df, x="date", y="count_queries", title="No. of Unique Queries Ranking in Google in USA")

fig.show()

In [ ]:
asdasd

# Grouped By Date : Count Queries Ranking in Top 10 Positions

In [ ]:
len(df)

1349633

In [ ]:
df_top_10_pos = df[df['position'] < 11 ]

In [ ]:
len(df_top_10_pos)

355252

In [ ]:
top_10_count_queries_df = df_top_10_pos.groupby("date").agg(
    clicks = ('clicks', 'sum'),
    impressions = ('impressions', 'sum'),
    position = ('position', 'mean'),
    count_queries = ('query', 'count')
    ).reset_index()

top_10_count_queries_df

,date,clicks,impressions,position,count_queries
0,2022-06-01,135,2645,5.288254,1076
1,2022-06-02,164,2709,5.134606,1090
2,2022-06-03,149,2440,5.243645,986
3,2022-06-04,79,1715,5.161339,871
4,2022-06-05,85,1830,5.378940,841
...,...,...,...,...,...
291,2023-03-19,58,2177,4.971404,985
292,2023-03-20,112,3214,4.890950,1264
293,2023-03-21,102,2983,4.909386,1293
294,2023-03-22,88,3224,5.070352,1286


In [ ]:
import plotly.express as px

fig = px.line(top_10_count_queries_df, x="date", y="count_queries", title="No. of Unique Queries Ranking in Top Positions in Google in USA")

fig.show()